# Applying BDT 

# Event processing

In [1]:
from root_pandas import read_root
import pandas as  pd
base = '/home/sutclw/Work/Analysis/b2ulnu/Data/B2GM_June_2019/20190516_BGM_2/'
df_clnu_md_gen = read_root(base +'generic/s1/tag_corr/bingen_md_s1_pre_selected_20190516_tagCorr.root','tree')
df_clnu_ch_gen = read_root(base +'generic/s1/tag_corr/bingen_ch_s1_pre_selected_20190516_tagCorr.root','tree')

Dlnu_ch_BGM_ffs = pd.read_pickle(base + 'generic/s1/clnu_FF/Dlnu_ch_BGM_s1_ffs.pickle')
Dlnu_md_BGM_ffs = pd.read_pickle(base + 'generic/s1/clnu_FF/Dlnu_md_BGM_s1_ffs.pickle')
Dstlnu_ch_BGM_ffs = pd.read_pickle(base + 'generic/s1/clnu_FF/Dstlnu_ch_BGM_s1_ffs.pickle')
Dstlnu_md_BGM_ffs = pd.read_pickle(base + 'generic/s1/clnu_FF/Dstlnu_md_BGM_s1_ffs.pickle')

df_dstst_ch = read_root(base + 'Dstst/Dstst_FFs/Dstst_ch_s1_pre_selected_20190516_FF.root', 'tree')
df_dstst_md = read_root(base + 'Dstst/Dstst_FFs/Dstst_md_s1_pre_selected_20190516_FF.root', 'tree')
df_gap_ch = read_root(base + 'Gap/s0/Gap_ch_s1_pre_selected_20190516_tagCorr.root', 'tree')
df_gap_md = read_root(base + 'Gap/s0/Gap_ch_s1_pre_selected_20190516_tagCorr.root', 'tree')
df_clnu_ch = pd.concat([df_clnu_ch_gen, Dlnu_ch_BGM_ffs, Dstlnu_ch_BGM_ffs], axis=1)
df_clnu_md = pd.concat([df_clnu_md_gen, Dlnu_md_BGM_ffs, Dstlnu_md_BGM_ffs], axis=1)
def mask_dstst_gap(df):
    
    xlv_decay = abs(df.event_lclass) ==2
    
    gap = xlv_decay & (df.event_dclass.isin([2,4,9,10]))
    dstst = xlv_decay & (df.event_dclass.isin([5,6,7,8]))
    # skip old gap and D**
    df_mask = df[~(gap | dstst)]
    return df_mask


df_clnu_md_mask = mask_dstst_gap(df_clnu_md)
df_clnu_ch_mask = mask_dstst_gap(df_clnu_ch)
del df_clnu_md, df_clnu_ch

df_clnu_md = pd.concat([df_clnu_md_mask, df_dstst_md, df_gap_md], ignore_index=True)
df_clnu_ch = pd.concat([df_clnu_ch_mask, df_dstst_ch, df_gap_ch], ignore_index=True)
DstFFcols = [col for col in  df_dstst_md.columns if 'FF'  in col]
list1 = list(Dlnu_ch_BGM_ffs.columns)+list(Dstlnu_ch_BGM_ffs)+DstFFcols
del(df_dstst_ch)
del(df_dstst_md)
del(df_gap_ch )

del df_clnu_md_mask, df_clnu_ch_mask
del(Dlnu_ch_BGM_ffs)
del(Dlnu_md_BGM_ffs)
dictionary = dict(zip(list1, len(list1)*[1]))
def TotalWeight(df,columns,newname):
    df[newname] = 1
    for col in columns:
        df[newname] = df[newname]*df[col]
df_clnu_md_filled =df_clnu_md.fillna(value=dictionary)
df_clnu_ch_filled=df_clnu_ch.fillna(value=dictionary)
del(df_clnu_md)
del(df_clnu_ch)
        

Welcome to JupyROOT 6.16/00


/home/sutclw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/sutclw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from BDT.EventPreprocessor import EventPreprocessor

base = '/home/sutclw/Work/Analysis/b2ulnu/Data/B2GM_June_2019/20190516_BGM_2/generic/s0/tag_corr/'
data = { 'charm' :base + 'bincharm_s0_pre_selected_20190516_tagCorr.root',
         'uds' : base + 'binuds_s0_pre_selected_20190516_tagCorr.root '}

# Do common event processing on datasets 
p = EventPreprocessor(data,'tree',usereadroot=True)
p.MultiplyWeights(['PID_w_0','ks_w_0','slowpi_w_0','tag_w_0'],'tot_w_0')
p.Combine(('uds','uds'),('charm','c'),'cont','BMeson')
p.AddNewColumnToAll('vertexfit', lambda df: np.where((df['gx_vtx_chi2']>0)&(df['gx_vtx_dgf']>0), np.log10(df.gx_vtx_chi2/df.gx_vtx_dgf), -100))
p.FillNaNs(10000)
p.AddNewColumnToX('_hybridweight_', lambda df: 1,'cont')



# Get data back from event processor split into ulnu and clnu 
cont = p.GetDataframe('cont')

# Split into B0 and B+
B0_cont_data = cont.query('((abs(btag_pcode_b) == 511))')
BP_cont_data = cont.query('((abs(btag_pcode_b) != 511))')

/home/sutclw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log10
  from ipykernel import kernelapp as app


# Load Weights and Apply

In [6]:
from importlib import reload 
import BDT
#reload(BDT.XGBoostInference)
from BDT.XGBoostInference import XGBoostInference
features = ['gmiss_m2','event_nlep', 'event_q','event_nk', 'event_nks', 'veto_slowCharPi_missM2', 'veto_slowNeuPi_missM2', 'vertexfit', 'veto_slowNeuPi_cos_ThetaC', 'veto_slowCharPi_cos_ThetaC', 'veto_slowCharPi_cos_BY', 'veto_slowNeuPi_cos_BY']
B0_inferer = XGBoostInference('mixed_BDT',features)
BP_inferer = XGBoostInference('charged_BDT',features)
B0_inferer.SetData(B0_cont_data)
B0_inferer.Apply()
BP_inferer.SetData(BP_cont_data)
BP_inferer.Apply()


# Save Data

In [7]:
from BDT.EventPreprocessor import EventPreprocessor
data = {}
data['cont_ch'] = BP_inferer.GetData()
data['cont_md'] = B0_inferer.GetData()
outp = EventPreprocessor(data,'tree')
outp.CombineMultiple(['cont_ch','cont_md'],'cont')
outp.ApplyOneMask("BMeson == 'c'",'cont','charm')
outp.ApplyOneMask("BMeson == 'uds'",'cont','uds')
outp.WriteFile('charm','charm_s0_BDT.root')
outp.WriteFile('uds','uds_s0_BDT.root')


In [7]:
df_clnu_ch_filled

NameError: name 'df_clnu_ch_filled' is not defined

In [2]:
import BDT
#reload(BDT.EventPreprocessor)
from BDT.EventPreprocessor import EventPreprocessor
import numpy as np
data = { 
        'clnu_ch' : df_clnu_ch_filled,
         'clnu_md' : df_clnu_md_filled}


# Do common event processing on datasets 
p = EventPreprocessor(data,'tree')
p.Combine(('clnu_ch','ch'),('clnu_md','md'),'clnu','BMeson')
p.MultiplyWeights(['PID_w_0',
 'ks_w_0',
 'slowpi_w_0',
 'tag_w_0',
 'Dlnu_FF_weight',
 'Dstlnu_FF_weight',
 'FF_Done_nominal',
 'FF_Dprimeone_nominal',
 'FF_Dtwostar_nominal',
 'FF_Dzerostar_nominal'],'tot_w_0')
p.AddNewColumnToAll('vertexfit', lambda df: np.where((df['gx_vtx_chi2']>0)&(df['gx_vtx_dgf']>0), np.log10(df.gx_vtx_chi2/df.gx_vtx_dgf), -100))
p.FillNaNs(10000)


# Get data back from event processor split into ulnu and clnu 

clnu = p.GetDataframe('clnu')

#Append data


# Split into B0 and B+
B0_data = clnu.query('((abs(btag_pcode_b) == 511))')
BP_data = clnu.query('((abs(btag_pcode_b) != 511))')


/home/sutclw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in log10


In [3]:
B0_data[np.isnan(B0_data['tot_w_0'])]

,Dgam_FF_downweight0,Dgam_FF_downweight1,Dgam_FF_downweight2,Dgam_FF_downweight3,Dgam_FF_downweight4,Dgam_FF_downweight5,Dgam_FF_upweight0,Dgam_FF_upweight1,Dgam_FF_upweight2,Dgam_FF_upweight3,...,veto_slowNeuPi_Pi_pc,veto_slowNeuPi_allPi_pc,veto_slowNeuPi_cos_BY,veto_slowNeuPi_cos_ThetaC,veto_slowNeuPi_missM2,veto_slowNeuPi_q2,BMeson,Index,tot_w_0,vertexfit


In [4]:
import BDT
#reload(BDT.XGBoostInference)
from BDT.XGBoostInference import XGBoostInference
features = ['gmiss_m2','event_nlep', 'event_q','event_nk', 'event_nks', 'veto_slowCharPi_missM2', 'veto_slowNeuPi_missM2', 'vertexfit', 'veto_slowNeuPi_cos_ThetaC', 'veto_slowCharPi_cos_ThetaC', 'veto_slowCharPi_cos_BY', 'veto_slowNeuPi_cos_BY']
B0_inferer = XGBoostInference('mixed_BDT',features)
BP_inferer = XGBoostInference('charged_BDT',features)
B0_inferer.SetData(B0_data)
B0_inferer.Apply()
BP_inferer.SetData(BP_data)
BP_inferer.Apply()

/home/sutclw/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/sutclw/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
from BDT.EventPreprocessor import EventPreprocessor

data = {}
data['clnu_ch'] = BP_inferer.GetData()
data['clnu_md'] = B0_inferer.GetData()
outp = EventPreprocessor(data,'tree')
outp.CombineMultiple(['clnu_ch','clnu_md'],'clnu')
outp.ApplyOneMask("BMeson == 'ch'",'clnu','clnu_charged')
outp.ApplyOneMask("BMeson == 'md'",'clnu','clnu_mixed')
outp.WriteFile('clnu_mixed','clnu_md_s1_BDT.root')
outp.WriteFile('clnu_charged','clnu_ch_s1_BDT.root')

/home/sutclw/anaconda3/lib/python3.7/site-packages/root_numpy/_tree.py:575: UserWarning: converter for dtype('O') is not implemented (skipping)
  cobj = _librootnumpy.array2tree_toCObj(arr, name=name, tree=incobj)


In [1]:
from root_pandas import read_root
df_data = read_root('/home/sutclw/Work/Analysis/b2ulnu/Data/B2GM_June_2019/20190516_BGM_2/data/pre_sel_generic/bindata_pre_selected_20190516.root')

Welcome to JupyROOT 6.16/00


In [2]:
import BDT
from BDT.EventPreprocessor import EventPreprocessor
import numpy as np
df_data['Index']= df_data.index
data = { 
        'data' : df_data
}
p = EventPreprocessor(data,'tree')

p.AddNewColumnToAll('vertexfit', lambda df: np.where((df['gx_vtx_chi2']>0)&(df['gx_vtx_dgf']>0), np.log10(df.gx_vtx_chi2/df.gx_vtx_dgf), -100))
p.FillNaNs(10000)


# Get data back from event processor split into ulnu and clnu 

data = p.GetDataframe('data')

#Append data


# Split into B0 and B+
B0_data = data.query('((abs(btag_pcode_b) == 511))')
BP_data = data.query('((abs(btag_pcode_b) != 511))')

/home/sutclw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in log10
  # Remove the CWD from sys.path while we load stuff.


In [3]:
import BDT
#reload(BDT.XGBoostInference)
from BDT.XGBoostInference import XGBoostInference
features = ['gmiss_m2','event_nlep', 'event_q','event_nk', 'event_nks', 'veto_slowCharPi_missM2', 'veto_slowNeuPi_missM2', 'vertexfit', 'veto_slowNeuPi_cos_ThetaC', 'veto_slowCharPi_cos_ThetaC', 'veto_slowCharPi_cos_BY', 'veto_slowNeuPi_cos_BY']
B0_inferer = XGBoostInference('mixed_BDT',features)
BP_inferer = XGBoostInference('charged_BDT',features)
B0_inferer.SetData(B0_data)
B0_inferer.Apply()
BP_inferer.SetData(BP_data)
BP_inferer.Apply()

/home/sutclw/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/sutclw/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [4]:
from BDT.EventPreprocessor import EventPreprocessor

data = {}
data['data_ch'] = BP_inferer.GetData()
data['data_md'] = B0_inferer.GetData()
outp = EventPreprocessor(data,'tree')
outp.CombineMultiple(['data_ch','data_md'],'datatotal')
outp.WriteFile('datatotal','data_BDT_20190516.root')


/home/sutclw/anaconda3/lib/python3.7/site-packages/root_numpy/_tree.py:575: UserWarning: converter for dtype('O') is not implemented (skipping)
  cobj = _librootnumpy.array2tree_toCObj(arr, name=name, tree=incobj)
